In [77]:
!pip install requests pandas mysql-connector-python SQLAlchemy



Defaulting to user installation because normal site-packages is not writeable


In [78]:
import requests
import pandas as pd

# API Configuration
API_URL = "https://8b1gektg00.execute-api.us-east-1.amazonaws.com/default/engineer-test"
API_KEY = "GSJ3bdgMaoa8bHOXGs5YQHRYxmHMSx96WUmbVAjj"  
HEADERS = {"x-api-key": API_KEY}

# Define the request payload for January 2023 data
payload = {
    "start_date": "2023-01-01",
    "end_date": "2023-01-31"
}

def fetch_transactions():
    """Fetch transaction data from the API."""
    response = requests.post(API_URL, json=payload, headers=HEADERS)
    
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data)  # Convert to Pandas DataFrame
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return pd.DataFrame()  # Return an empty DataFrame if request fails

# Fetch the data
df_raw = fetch_transactions()

# Display the first few rows
df_raw.head()


,customer_id,product_id,transaction_date,transaction_amount,transaction_type,spend_category,product_category
0,4,32,2023-01-01 13:07:24,168.44,Outgoing,Electronics,NaN
1,6,32,2023-01-01 15:56:07,832.92,Outgoing,Electronics,NaN
2,7,37,2023-01-01 03:08:20,67.06,Outgoing,Clothing,NaN
3,3,34,2023-01-01 04:06:55,333.12,Outgoing,None,NaN
4,4,36,2023-01-01 11:30:27,348.90,Outgoing,Home,NaN


In [79]:
print(df_raw)

      customer_id  product_id     transaction_date  transaction_amount  \
0               4          32  2023-01-01 13:07:24              168.44   
1               6          32  2023-01-01 15:56:07              832.92   
2               7          37  2023-01-01 03:08:20               67.06   
3               3          34  2023-01-01 04:06:55              333.12   
4               4          36  2023-01-01 11:30:27              348.90   
...           ...         ...                  ...                 ...   
1796            5          40  2023-01-31 23:36:24              290.30   
1797            2          34  2023-01-31 05:28:53              283.99   
1798           10          35  2023-01-31 12:37:06              519.73   
1799            7          35  2023-01-31 16:18:26              558.23   
1800            2          38  2023-01-31 16:26:37              761.12   

     transaction_type spend_category product_category  
0            Outgoing    Electronics              NaN  

In [80]:
# Convert 'transaction_date' to datetime format
df_raw['transaction_date'] = pd.to_datetime(df_raw['transaction_date'], errors='coerce')

# Drop rows where transaction_date is NaT (invalid date format)
df_raw = df_raw.dropna(subset=['transaction_date'])

# Drop duplicates
df_raw = df_raw.drop_duplicates()

# Filter out negative transaction amounts
df_cleaned = df_raw[df_raw['transaction_amount'] >= 0]

#Fix null values in product category by taking values from spend_category where it is null
df_cleaned['product_category'] = df_cleaned['product_category'].fillna(df_cleaned['spend_category'])

# Display cleaned data
df_cleaned.head()


C:\Users\User\AppData\Local\Temp\ipykernel_22352\1548652935.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['product_category'] = df_cleaned['product_category'].fillna(df_cleaned['spend_category'])


,customer_id,product_id,transaction_date,transaction_amount,transaction_type,spend_category,product_category
0,4,32,2023-01-01 13:07:24,168.44,Outgoing,Electronics,Electronics
1,6,32,2023-01-01 15:56:07,832.92,Outgoing,Electronics,Electronics
2,7,37,2023-01-01 03:08:20,67.06,Outgoing,Clothing,Clothing
3,3,34,2023-01-01 04:06:55,333.12,Outgoing,None,None
4,4,36,2023-01-01 11:30:27,348.90,Outgoing,Home,Home


In [81]:
def categorize_amount(amount):
    """Categorize transaction amount."""
    if amount < 50:
        return "Low"
    elif 50 <= amount <= 200:
        return "Medium"
    else:
        return "High"

# Apply categorization
df_cleaned['transaction_category'] = df_cleaned['transaction_amount'].apply(categorize_amount)


C:\Users\User\AppData\Local\Temp\ipykernel_22352\1471277612.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['transaction_category'] = df_cleaned['transaction_amount'].apply(categorize_amount)


In [82]:
# Create a full independent copy
df_transformed = df_cleaned.copy()

# Apply categorization
df_transformed['transaction_category'] = df_transformed['transaction_amount'].apply(categorize_amount)

# Display the transformed DataFrame
df_transformed.head()


,customer_id,product_id,transaction_date,transaction_amount,transaction_type,spend_category,product_category,transaction_category
0,4,32,2023-01-01 13:07:24,168.44,Outgoing,Electronics,Electronics,Medium
1,6,32,2023-01-01 15:56:07,832.92,Outgoing,Electronics,Electronics,High
2,7,37,2023-01-01 03:08:20,67.06,Outgoing,Clothing,Clothing,Medium
3,3,34,2023-01-01 04:06:55,333.12,Outgoing,None,None,High
4,4,36,2023-01-01 11:30:27,348.90,Outgoing,Home,Home,High


In [83]:
# Calculate total transaction value per customer
customer_totals = df_transformed.groupby("customer_id")["transaction_amount"].sum().reset_index()
customer_totals.rename(columns={"transaction_amount": "total_transaction_value"}, inplace=True)

# Merge back into the transformed dataset
df_final = df_transformed.merge(customer_totals, on="customer_id", how="left")

# Display transformed data
df_final.head()


,customer_id,product_id,transaction_date,transaction_amount,transaction_type,spend_category,product_category,transaction_category,total_transaction_value
0,4,32,2023-01-01 13:07:24,168.44,Outgoing,Electronics,Electronics,Medium,"274,222.92"
1,6,32,2023-01-01 15:56:07,832.92,Outgoing,Electronics,Electronics,High,"387,381.73"
2,7,37,2023-01-01 03:08:20,67.06,Outgoing,Clothing,Clothing,Medium,"97,124,599.01"
3,3,34,2023-01-01 04:06:55,333.12,Outgoing,None,None,High,"124,238,969.54"
4,4,36,2023-01-01 11:30:27,348.90,Outgoing,Home,Home,High,"274,222.92"


In [84]:
pd.options.display.float_format = '{:,.2f}'.format  # Displays numbers with commas & 2 decimal places
print(df_final.head())


   customer_id  product_id    transaction_date  transaction_amount  \
0            4          32 2023-01-01 13:07:24              168.44   
1            6          32 2023-01-01 15:56:07              832.92   
2            7          37 2023-01-01 03:08:20               67.06   
3            3          34 2023-01-01 04:06:55              333.12   
4            4          36 2023-01-01 11:30:27              348.90   

  transaction_type spend_category product_category transaction_category  \
0         Outgoing    Electronics      Electronics               Medium   
1         Outgoing    Electronics      Electronics                 High   
2         Outgoing       Clothing         Clothing               Medium   
3         Outgoing           None             None                 High   
4         Outgoing           Home             Home                 High   

   total_transaction_value  
0               274,222.92  
1               387,381.73  
2            97,124,599.01  
3           

In [85]:
!pip install mysql-connector-python



Defaulting to user installation because normal site-packages is not writeable


In [100]:
# Convert 'transaction_date' to datetime type
df_final['transaction_date'] = pd.to_datetime(df_final['transaction_date'], errors='coerce')

# Convert 'transaction_date' to MySQL-compatible format (string format)
df_final['transaction_date'] = df_final['transaction_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Establish connection to MySQL
import mysql.connector

conn = mysql.connector.connect(
    host="127.0.0.1",  # e.g., 'localhost' or an IP address
    user="root",
    password="Assesment!23",
    database="transactions_db"
)

cursor = conn.cursor()

# Insert transformed data into MySQL(Used INSERT IGNORE to Skip Duplicates when loading into target continuously)
insert_query = """
INSERT IGNORE INTO Customer.Transactions (customer_id, product_id, transaction_amount, transaction_date, transaction_category, total_transaction_value, product_category, spend_category)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Loop through the DataFrame and insert data row by row
try:
    for index, row in df_final.iterrows():
        cursor.execute(insert_query, (row['customer_id'], row['product_id'], row['transaction_amount'], row['transaction_date'],
                                      row['transaction_category'], row['total_transaction_value'], row['product_category'], row['spend_category']))

    # Commit the changes
    conn.commit()
    print("Data successfully loaded into MySQL!")

except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()  # Rollback in case of error
finally:
    cursor.close()
    conn.close()


Data successfully loaded into MySQL!
